# Revised case normalization for USZ 2019

Notes:

- 25 of 25 revised DtoD cases were joined with the BFS cases from the DB.


In [1]:
import sys

import pandas as pd
import numpy as np

from loguru import logger

sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.py.global_configs import *
from src.revised_case_normalization.py.normalize import normalize
from src.service import bfs_cases_db_service as bfs_db
from src.service.bfs_cases_db_service import session, get_hospital_year_cases

In [2]:
file_info = FileInfo(
        os.path.join(ROOT_DIR, 'raw_data/KSSG_2021.xlsx'),
        'KSSG HQ',
        '2021',
        ['Änderungen_KSSG_2021'])

print(file_info)

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/KSSG_2021.xlsx', hospital_name_db='KSSG HQ', year='2021', sheets=['Änderungen_KSSG_2021'])


In [3]:
cols_to_rename = dict(COLUMNS_TO_RENAME)
# Replace 'admno' with 'fall nummer'
cols_to_rename.pop('admno')
cols_to_rename['fall nummer'] = CASE_ID_COL

df_revised_case_d2d = normalize(file_info, 0)

2022-10-19 16:22:54.350 | INFO     | src.revised_case_normalization.py.normalize:normalize:46 - Read 25 cases for KSSG HQ 2021
2022-10-19 16:22:54.354 | INFO     | src.revised_case_normalization.py.normalize:normalize:69 - TYPES:
case_id              int64
patient_id          object
gender              string
age_years            int64
duration_of_stay     int64
pccl                 int64
drg                 string
old_pd              string
new_pd              string
bfs_code            string
added_icds          string
removed_icds        string
added_chops         string
removed_chops       string
dtype: object
2022-10-19 16:22:54.359 | INFO     | src.utils.dataframe_utils:validate_icd_codes:68 - Validated ICD codes in 'added_icds' and stored them into 'added_icds': All rows contain valid codes
2022-10-19 16:22:54.361 | INFO     | src.utils.dataframe_utils:validate_icd_codes:68 - Validated ICD codes in 'removed_icds' and stored them into 'removed_icds': All rows contain valid codes


In [4]:
df_revised_case_d2d.head()

,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,old_pd,new_pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops
0,1112425128,F1B1A46BC1FEF112,M,56,5,3,H41C,K831,K8031,M100,"[K8510, D6959]",[],[],[]
1,1112427844,59106C27B3536BE5,M,64,9,3,T62C,R5088,R5088,M100,"[I691, G819]",[],[],[]
2,1112441515,BB0033F7EF9CCB1F,W,70,3,3,R61D,C833,C833,M100,[D6110],[],[],[]
3,1112445586,73EFDADADF92D6A4,W,93,7,2,F62D,I5001,I5001,M100,[F050],[],[],[]
4,1112426216,FD14EA356F1A32E4,M,78,4,2,E77E,J128,J128,M100,[J4412],[J4482],[],[]


# Match revised DtoD data with the BfS data from the database

In [5]:
cases_in_db = get_hospital_year_cases(file_info.hospital_name_db, file_info.year)[[AIMEDIC_ID] + VALIDATION_COLS]
cases_in_db.head()


,aimedic_id,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg
0,455133,1111866085,39F9EA99CEA02FF8,M,69,14,4,F01E
1,455134,1111866101,330A58EDEB982EF2,M,53,2,0,X64Z
2,455135,1111866105,BBFDA514D2A94FF7,M,75,7,3,B66C
3,455136,1111866107,30B91142C7B95FDF,M,80,17,4,W61A
4,455137,1111866108,4165057BE3392CE0,M,28,3,0,E06C


In [6]:
num_cases_in_db = cases_in_db.shape[0]
if num_cases_in_db == 0:
    raise ValueError(f"There is no data for the hospital '{file_info.hospital_name_db}' in {file_info.year}")
else:
    logger.info(f"Read {num_cases_in_db} rows from the DB, for the hospital '{file_info.hospital_name_db}' in {file_info.year}")    

2022-10-19 16:23:21.944 | INFO     | __main__:<module>:5 - Read 28977 rows from the DB, for the hospital 'KSSG HQ' in 2021


In [7]:
# Remove columns with empty 'case_id' in DB data

cases_in_db['case_id'].replace('', np.nan, inplace=True)
num_na_cases_in_db = cases_in_db['case_id'].isna().sum()
cases_in_db.dropna(subset=['case_id'], inplace=True)
logger.info(f"{num_na_cases_in_db} rows with empty 'case_id's were discovered and removed from the DB, for the hospital '{file_info.hospital_name_db}' in {file_info.year}")   

2022-10-19 16:23:21.965 | INFO     | __main__:<module>:6 - 0 rows with empty 'case_id's were discovered and removed from the DB, for the hospital 'KSSG HQ' in 2021


In [8]:
# get the case_id from revised_case 
df_revised_case_d2d['case_id_num'] = df_revised_case_d2d[CASE_ID_COL].astype(int)
cases_in_db['case_id_num'] = cases_in_db[CASE_ID_COL].astype(int)

In [9]:
# get case ids in revised case data from DtoD
case_id_d2d = df_revised_case_d2d['case_id_num'].values

In [10]:
# Check for duplicates in the case ids of DtoD

case_id_d2d_set = set(case_id_d2d)
if len(case_id_d2d) > len(case_id_d2d_set):
    logger.info('There are duplicated case_ids in the revised cases.' )
else:
    logger.info('There are no duplicated case_ids in the revised cases.' )

len(case_id_d2d_set)

2022-10-19 16:23:21.991 | INFO     | __main__:<module>:7 - There are no duplicated case_ids in the revised cases.


25

In [11]:
# Search for case_ids in the BFS DB data with the same case_ids as in revised case dataset of DtoD 
# Reason: check whether case ids of the revised data appear more than once in BFS DB data

revised_case_bfs_db = cases_in_db[cases_in_db['case_id_num'].isin(case_id_d2d)]
revised_case_bfs_db.shape[0]

25

In [12]:
# Join revised cases from DtoD with BFS Cases from Database

if revised_case_bfs_db.shape[0] <= len(case_id_d2d):
    revised_case_d2d_subset = df_revised_case_d2d[COL_SUBSET_FROM_REVISED_CASES + ['case_id_num', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg']]
    revised_case_normalized = revised_case_d2d_subset.merge(revised_case_bfs_db, how ='inner', on=['case_id_num', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg'], suffixes=None)
    num_match_case_id = revised_case_normalized.shape[0]
else:
    revised_case_d2d_subset = df_revised_case_d2d[COL_SUBSET_FROM_REVISED_CASES + ['case_id_num', 'patient_id', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg']]
    revised_case_normalized = revised_case_d2d_subset.merge(revised_case_bfs_db, how ='inner', on=['case_id_num', 'patient_id', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg'])
    num_match_case_patient_id = revised_case_normalized.shape[0]
print(revised_case_normalized.shape)
# revised_case_normalized.head()

(25, 16)


/tmp/ipykernel_20456/3898438272.py:5: FutureWarning: Passing 'suffixes' as a <class 'NoneType'>, is not supported and may give unexpected results. Provide 'suffixes' as a tuple instead. In the future a 'TypeError' will be raised.
  revised_case_normalized = revised_case_d2d_subset.merge(revised_case_bfs_db, how ='inner', on=['case_id_num', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg'], suffixes=None)


In [13]:
# Join the revised cases of DtoD with the BFS data (alternative solution to cross check results)

#if revised_case_bfs_db.shape[0] <= len(case_id_d2d):
#    revised_case_normalized = pd.merge(df_revised_case_d2d, cases_in_db, suffixes=('_drop', ''), on=['case_id_num', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg'])
#    revised_case_normalized.drop([i for i in revised_case_normalized.columns if '_drop' in i], axis=1, inplace=True)
#    revised_case_normalized.head()
#else:
#    revised_case_normalized = pd.merge(df_revised_case_d2d, cases_in_db, suffixes=('_drop', ''), on=['case_id_num','patient_id', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg'])
#    revised_case_normalized.drop([i for i in revised_case_normalized.columns if '_drop' in i], axis=1, inplace=True)
#    revised_case_normalized.head()
#print(revised_case_normalized.shape)
#revised_case_normalized.head()

In [14]:
num_revised_case_normalized = revised_case_normalized.shape[0]
num_df_revised_case_d2d = df_revised_case_d2d.shape[0] 

if num_revised_case_normalized == 0:
    raise ValueError(f"No cases between the revised cases for the hospital '{file_info.hospital_name_db}' in {file_info.year} were joined.")
else:
    logger.info(f"{num_revised_case_normalized} of {num_df_revised_case_d2d} rows were joined with the cases from the DB, for the hospital '{file_info.hospital_name_db}' in {file_info.year}") 

2022-10-19 16:23:22.027 | INFO     | __main__:<module>:7 - 25 of 25 rows were joined with the cases from the DB, for the hospital 'KSSG HQ' in 2021


In [15]:
df_revised_case_normalized = revised_case_normalized[COLS_TO_OUTPUT]

In [16]:
# Export normalized revision data as a csv: 

hospital = file_info.hospital_name_db.replace(' ', '_') # replace spaces with '_' for the file name
hospital_year = f'{hospital}_{file_info.year}'

output_folder = os.path.join(ROOT_DIR, 'normalized_revision_data/')

if not os.path.exists(output_folder):
    os.mkdir(output_folder)

NEW = '_NEW'
    
output_path = os.path.join(ROOT_DIR, 'normalized_revision_data/') + hospital_year + NEW + '.csv'

df_revised_case_normalized.sort_values('aimedic_id').to_csv(output_path)


